In [ ]:
!pip install azure-storage-blob azure-identity --quiet
!pip install tensorflow-io --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 9.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/harvard.dce.nasa.cloud2cloud

Mounted at /content/drive
/content/drive/MyDrive/harvard.dce.nasa.cloud2cloud


In [ ]:
from google.colab import userdata
import os

account_name = userdata.get('storage_account_name')
account_key = userdata.get('storage_account_key')
container_name = userdata.get('blob_container_name')

os.environ['TF_AZURE_STORAGE_KEY'] = account_key

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

# more info https://learn.microsoft.com/en-us/python/api/overview/azure/storage-blob-readme?view=azure-python
connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"

# setup to load file from blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
%cd capstone_2024/cloud_features_tracking/
import cloud2cloud_ConvNext as CN

/content/drive/MyDrive/harvard.dce.nasa.cloud2cloud/capstone_2024/cloud_features_tracking


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import AutoImageProcessor, ConvNextModel
from PIL import Image
import tensorflow as tf
import tensorflow_io as tfio
from azure.storage.blob import BlobServiceClient


def load_image_from_blob_cv(blob_img, container_client):
    """
    Loads the image from Azure Blob Storage using OpenCV and returns it as a numpy array.
    Args:
        blob_img (str): name of the blob image in the container
        container_client (azure.storage.blob.BlobContainerClient): container client
    Returns:
        (numpy.ndarray): loaded image with original channels retained
    """
    blob_client = container_client.get_blob_client(blob_img)
    streamdownloader = blob_client.download_blob()
    blob_data = streamdownloader.readall()
    image_array = np.asarray(bytearray(blob_data), dtype=np.uint8)
    # img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img_bgr = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb


def undistort_fisheye_image(distorted_image):
    """
    Apply correction for fisheye distortion.
    Args:
        distorted_image (numpy.ndarray): original image that has the fisheye distortion.
     Returns:
        (numpy.ndarray): undistorted image with fisheye correction.
    """
    # Parameters provided
    f = 1.4  # focal length [mm]
    mu = 2.8e-3  # pixel pitch [mm]
    S = 2  # output (undistorted) image scale factor
    # distortion polynomial order:  [2 4 6 8]
    # polynomial coefficients:
    coeffs = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    H = distorted_image.shape[0]  # image height [pixel]
    W = distorted_image.shape[1]  # image width [pixel]
    cx = (W - 1) / 2  # image center coordinate [pixel]
    cy = (H - 1) / 2  # image center coordinate [pixel]

    K = np.array([[f / mu, 0, cx], [0, f / mu, cy], [0, 0, 1]])

    # compute intrinsic matrix for undistorted image
    cpx = (W * S - 1) / 2
    cpy = (H * S - 1) / 2
    P = np.array([[f / mu, 0, cpx], [0, f / mu, cpy], [0, 0, 1]])

    # rectification matrix (identity)
    R = np.eye(3)

    # produce undistorted image
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K=K, D=coeffs, R=R, P=P, size=[W * S, H * S], m1type=cv2.CV_16SC2)
    undistorted_image = cv2.remap(distorted_image, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)
    return undistorted_image


def crop_and_correct_image_cv(image, size1=(1500, 1500), offset=(85, 180), size2=(1000, 1000)):
    h, w = image.shape[:2]

    # Cropping it to 1500X1500 and accounting for offset.
    center_h, center_w = h // 2, w // 2
    offset_h, offset_w = offset
    start_h = min(max(center_h - size1[0] // 2 + offset_h, 0), h - size1[0])
    start_w = min(max(center_w - size1[1] // 2 + offset_w, 0), w - size1[1])
    cropped_image = image[start_h:start_h + size1[0], start_w:start_w + size1[1]]
    h, w = cropped_image.shape[:2]

    # further crop the image to 1000X1000
    center_h, center_w = h // 2, w // 2
    start_h = max(center_h - size2[0] // 2, 0)
    start_w = max(center_w - size2[1] // 2, 0)

    return cropped_image[start_h:start_h + size2[0], start_w:start_w + size2[1]]

def crop_and_correct_image_cv2(image, size1=(1500, 1500), offset=(85, 180), size2=(800, 800)):
    """
    Crops the undistorted image in two stages and ensures the original center (960, 540)
    aligns with the center of the final cropped image (400, 400).

    Args:
        image (numpy.ndarray): Input image to crop.
        size1 (tuple): Size of the first crop (width, height).
        offset (tuple): Offset for the first crop.
        size2 (tuple): Size of the final crop (width, height).

    Returns:
        numpy.ndarray: Cropped image.
    """
    h, w = image.shape[:2]

    # Step 1: First crop with offset
    center_h, center_w = h // 2, w // 2  # Center of the undistorted image
    offset_h, offset_w = offset

    # Adjust the starting coordinates for the first crop
    start_h1 = max(center_h - size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - size1[0] // 2 + offset_w, 0)
    cropped_image = image[start_h1:start_h1 + size1[1], start_w1:start_w1 + size1[0]]

    # Step 2: Adjust second crop to ensure the original center aligns with the center of the crop-corrected image
    crop_h, crop_w = size2
    center_h_crop = center_h - start_h1  # Adjusted center in the cropped image
    center_w_crop = center_w - start_w1

    # Calculate start coordinates to place the center at 400, 400
    start_h2 = max(center_h_crop - crop_h // 2, 0)
    start_w2 = max(center_w_crop - crop_w // 2, 0)

    final_image = cropped_image[start_h2:start_h2 + crop_h, start_w2:start_w2 + crop_w]

    return final_image



def save_cropped_image(container_client, cropped_img, src_blob_name, dest):
    """
    Saves the cropped image to Azure Blob Storage with a modified folder name using OpenCV.

    Args:
        container_client (azure.storage.blob.ContainerClient): Azure blob storage container client
        cropped_img (numpy.ndarray): Cropped image as a numpy array.
        src_blob_name (str): Source blob name.
        dest (str): The folder where cropped images will be saved.

    Returns:
        None
    """
    cropped_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    is_success, buffer = cv2.imencode('.jpg', cropped_img_rgb)
    if not is_success:
        raise ValueError("Failed to encode image with OpenCV.")

    img_byte_array = BytesIO(buffer)

    base_name = os.path.basename(src_blob_name)
    cropped_blob_name = os.path.join(dest, base_name)

    blob_client = container_client.get_blob_client(cropped_blob_name)
    blob_client.upload_blob(img_byte_array.getvalue(), overwrite=True)


def crop_and_correct_images_in_blob(container_client, src, dest):
    """
    Reads images from src dir in an azure blob, applies crop_to_square,
    and uploads cropped images to dest dir in the same blob.

    Args:
        container_client (azure.storage.blob.ContainerClient): azure blob storage container client
        src (str): path to the dir containing src images.
        dest (str): path to the dir with cropped images.

    Returns:
        (str): path to the cropped images directory.
    """
    blobs = container_client.list_blobs(name_starts_with=src)
    existing_blobs = {blob.name for blob in container_client.list_blobs(name_starts_with=dest)}
    for blob in blobs:
        dir = os.path.dirname(blob.name)
        if dir.endswith('_frames') and blob.name.endswith('.jpg'):
            base_name = os.path.basename(blob.name)
            new_filename = base_name.replace('.jpg', '_crop_corrected.jpg')
            cropped_blob_name = os.path.join(dest, new_filename)
            if cropped_blob_name in existing_blobs:
                continue
            img_stream = load_image_from_blob_cv(blob.name, container_client)
            fisheye_corrected_img = undistort_fisheye_image(img_stream)
            cropped_img = crop_and_correct_image_cv(fisheye_corrected_img)

            save_cropped_image(container_client, cropped_img, blob.name, dest)
    return dest

In [ ]:
def map_coordinates(coord, original_size=(1920, 1080), fisheye_params=None, crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(1000, 1000)):
    """
    Maps a coordinate from the original image to the undistorted image and the crop-corrected image.

    Args:
        coord (tuple): (x, y) coordinate in the original image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye undistortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and crop-corrected coordinates.
    """
    x, y = coord

    # Map from original to undistorted image
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        R = fisheye_params.get("R", np.eye(3))
        P = fisheye_params["P"]

        # Normalize coordinates to camera space
        coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
        undistorted_coord = cv2.fisheye.undistortPoints(
            coord_homog.T[:, :2].reshape(1, -1, 2),
            K=K, D=D, R=R, P=P
        )[0][0]
        x_undist, y_undist = undistorted_coord
    else:
        x_undist, y_undist = x, y

    # Map from undistorted to crop-corrected image (first crop)
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop coordinates
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)
    x_cropped1 = x_undist - start_w1
    y_cropped1 = y_undist - start_h1

    # Map from first cropped image to final crop-corrected image
    crop1_h, crop1_w = crop_size1
    center_h1, center_w1 = crop1_h // 2, crop1_w // 2

    # Adjust for alignment to center (500, 500)
    start_h2 = center_h1 - crop_size2[1] // 2
    start_w2 = center_w1 - crop_size2[0] // 2
    x_cropped2 = x_cropped1 - start_w2
    y_cropped2 = y_cropped1 - start_h2

    # Return both mapped coordinates
    return {
        "undistorted": (x_undist, y_undist),
        "crop_corrected": (x_cropped2, y_cropped2)
    }

def map_coordinates2(coord, original_size=(1920, 1080), fisheye_params=None, crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(800, 800)):
    """
    Maps a coordinate from the original image to the undistorted image and the crop-corrected image.

    Args:
        coord (tuple): (x, y) coordinate in the original image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye undistortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and crop-corrected coordinates.
    """
    x, y = coord

    # Step 1: Map from original to undistorted image
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        R = fisheye_params.get("R", np.eye(3))
        P = fisheye_params["P"]

        # Normalize coordinates to camera space
        coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
        undistorted_coord = cv2.fisheye.undistortPoints(
            coord_homog.T[:, :2].reshape(1, -1, 2),
            K=K, D=D, R=R, P=P
        )[0][0]
        x_undist, y_undist = undistorted_coord
    else:
        x_undist, y_undist = x, y  # No undistortion applied

    # Step 2: Map from undistorted to first cropped image
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop starting points
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)
    x_cropped1 = x_undist - start_w1
    y_cropped1 = y_undist - start_h1

    # Step 3: Map from first cropped image to final crop-corrected image
    center_h_crop = center_h - start_h1  # Adjusted center in the first cropped image
    center_w_crop = center_w - start_w1

    # Adjust coordinates for final cropping stage
    start_h2 = max(center_h_crop - crop_size2[1] // 2, 0)
    start_w2 = max(center_w_crop - crop_size2[0] // 2, 0)
    x_cropped2 = x_cropped1 - start_w2
    y_cropped2 = y_cropped1 - start_h2

    # Return the coordinates for both stages
    return {
        "undistorted": (x_undist, y_undist),
        "crop_corrected": (x_cropped2, y_cropped2)
    }

def unmap_coordinates2(crop_corrected_coord, original_size=(1920, 1080), fisheye_params=None,
                       crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(800, 800)):
    """
    Reverses the mapping from crop-corrected coordinates to the undistorted and original image coordinates.

    Args:
        crop_corrected_coord (tuple): (x, y) coordinate in the crop-corrected image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye distortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and original coordinates.
    """
    x_cropped2, y_cropped2 = crop_corrected_coord

    # Step 1: Reconstruct the undistorted coordinates
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop starting points
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)

    # Adjusted center in the first cropped image
    center_h_crop = center_h - start_h1
    center_w_crop = center_w - start_w1

    # Second crop starting points
    start_h2 = max(center_h_crop - crop_size2[1] // 2, 0)
    start_w2 = max(center_w_crop - crop_size2[0] // 2, 0)

    # Reconstruct undistorted coordinates
    x_cropped1 = x_cropped2 + start_w2
    y_cropped1 = y_cropped2 + start_h2
    x_undist = x_cropped1 + start_w1
    y_undist = y_cropped1 + start_h1

    # Step 2: Adjust undistorted coordinates to match the original camera matrix
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        P = fisheye_params["P"]

        # Compute the offset between the principal points of K and P
        cx_offset = P[0, 2] - K[0, 2]
        cy_offset = P[1, 2] - K[1, 2]

        # Adjust the undistorted coordinates
        x_undist_adjusted = x_undist - cx_offset
        y_undist_adjusted = y_undist - cy_offset

        # Normalize the adjusted undistorted coordinates
        x_norm_undist = (x_undist_adjusted - K[0, 2]) / K[0, 0]
        y_norm_undist = (y_undist_adjusted - K[1, 2]) / K[1, 1]

        # Prepare undistorted normalized coordinates
        undistorted_points = np.array([[[x_norm_undist, y_norm_undist]]], dtype=np.float64)

        # Apply fisheye distortion to get normalized distorted coordinates
        distorted_points = cv2.fisheye.distortPoints(
            undistorted_points,
            K=np.eye(3),
            D=D
        )

        # Map normalized distorted coordinates back to pixel coordinates using K
        x_distorted = distorted_points[0, 0, 0] * K[0, 0] + K[0, 2]
        y_distorted = distorted_points[0, 0, 1] * K[1, 1] + K[1, 2]
        x, y = x_distorted, y_distorted
    else:
        x, y = x_undist, y_undist  # No distortion applied

    # Return the coordinates for both stages
    return {
        "undistorted": (x_undist, y_undist),
        "original": (x, y)
    }


In [ ]:
import numpy as np

def get_fisheye_params(image_width, image_height, f=1.4, mu=2.8e-3, S=2):
    """
    Compute fisheye parameters (K, D, R, P) based on the given image dimensions and parameters.

    Args:
        image_width (int): Width of the original image.
        image_height (int): Height of the original image.
        f (float): Focal length in mm.
        mu (float): Pixel pitch in mm/pixel.
        S (float): Scaling factor for the undistorted image.

    Returns:
        dict: Dictionary containing K, D, R, and P matrices.
    """
    # Intrinsic matrix for the original image
    cx = (image_width - 1) / 2
    cy = (image_height - 1) / 2
    K = np.array([
        [f / mu, 0, cx],
        [0, f / mu, cy],
        [0, 0, 1]
    ])

    # Distortion coefficients
    D = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    # Rectification matrix (identity)
    R = np.eye(3)

    # Projection matrix for the undistorted image
    cpx = (image_width * S - 1) / 2
    cpy = (image_height * S - 1) / 2
    P = np.array([
        [f / mu, 0, cpx],
        [0, f / mu, cpy],
        [0, 0, 1]
    ])

    return {
        "K": K,
        "D": D,
        "R": R,
        "P": P
    }

# Parameters for the original image size (1920x1080)
fisheye_params = get_fisheye_params(image_width=1920, image_height=1080)

# Output the parameters
print("Intrinsic Matrix (K):\n", fisheye_params["K"])
print("Distortion Coefficients (D):\n", fisheye_params["D"])
print("Rectification Matrix (R):\n", fisheye_params["R"])
print("Projection Matrix (P):\n", fisheye_params["P"])


Intrinsic Matrix (K):
 [[500.    0.  959.5]
 [  0.  500.  539.5]
 [  0.    0.    1. ]]
Distortion Coefficients (D):
 [ 0.01166363 -0.04819808  0.07918044 -0.037572  ]
Rectification Matrix (R):
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Projection Matrix (P):
 [[5.0000e+02 0.0000e+00 1.9195e+03]
 [0.0000e+00 5.0000e+02 1.0795e+03]
 [0.0000e+00 0.0000e+00 1.0000e+00]]


In [ ]:
four_corners_and_centre = [(960, 540), (710,290), (1210,290), (710,790), (1210,790)]

mapped_coords = []
for point in four_corners_and_centre:
    print(map_coordinates(coord=point, fisheye_params=fisheye_params))
    print("\n")

{'undistorted': (1920.0, 1080.0), 'crop_corrected': (320.0, 415.0)}


{'undistorted': (1618.8156, 778.8155), 'crop_corrected': (18.8155517578125, 113.81549072265625)}


{'undistorted': (2221.646, 778.56024), 'crop_corrected': (621.64599609375, 113.56024169921875)}


{'undistorted': (1618.5602, 1381.646), 'crop_corrected': (18.5601806640625, 716.64599609375)}


{'undistorted': (2221.9028, 1381.9027), 'crop_corrected': (621.90283203125, 716.9027099609375)}




In [ ]:
originals = [
    "20170418/170418_175706_183328_frames/20170418_175706_frame_0.jpg",
    "20170418/170418_175706_183328_frames/20170418_175707_frame_60.jpg",
    "20170418/170418_175706_183328_frames/20170418_175708_frame_120.jpg",
    "20170418/170418_175706_183328_frames/20170418_175709_frame_180.jpg",
    "20170418/170418_175706_183328_frames/20170418_175710_frame_240.jpg"
]

original_images = [load_image_from_blob_cv(original, container_client) for original in originals]
fisheye_crr_images = [undistort_fisheye_image(original_image) for original_image in original_images]
crop_corrected_images = [crop_and_correct_image_cv2(corrected_image) for corrected_image in fisheye_crr_images]
grey_scale_images = [cv2.cvtColor(crop_corr_image, cv2.COLOR_RGB2GRAY) for crop_corr_image in crop_corrected_images]
augmented_images = [CN.augment_greyscale_image(grey_scale_image, contrast_factor=1.5, brightness_beta=30, kernel_size=(5, 5)) for grey_scale_image in grey_scale_images]
augmented_gbr_images = [cv2.cvtColor(cv2.cvtColor(augmented_image, cv2.COLOR_GRAY2RGB), cv2.COLOR_RGB2BGR) for augmented_image in augmented_images]

In [ ]:
four_corners_and_centre = [(960, 540),
                (710,290),
                (1210,290),
                (710,790),
                (1210,790)]

mapped_coords = []
for point in four_corners_and_centre:
    mapped_coords.append(map_coordinates2(coord=point, fisheye_params=fisheye_params))
print(mapped_coords)

fig, axes = plt.subplots(len(originals), 6, figsize=(20, 20))

for i, (original_image, fisheye_image, crop_corrected_image, grey_scale_image, augmented_image, augmented_gbr_image) in enumerate(zip(original_images, fisheye_crr_images, crop_corrected_images, grey_scale_images, augmented_images, augmented_gbr_images)):
    # Map coordinates to each image stage
    axes[i, 0].imshow(original_image)
    for point in four_corners_and_centre:
      axes[i, 0].plot(point[0], point[1], 'ro')
    axes[i, 0].set_title(f'original_image {i+1}')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(fisheye_image)
    for point in mapped_coords:
      axes[i, 1].plot(point["undistorted"][0], point["undistorted"][1], 'ro')
    axes[i, 1].set_title(f'undistorted_image {i+1}')
    axes[i, 1].axis('off')

    axes[i, 2].imshow(crop_corrected_image)
    for point in mapped_coords:
      axes[i, 2].plot(point["crop_corrected"][0], point["crop_corrected"][1], 'ro')
    axes[i, 2].set_title(f'crop_corrected_image {i+1}')
    axes[i, 2].axis('off')

    axes[i, 3].imshow(grey_scale_image, cmap='gray')
    for point in mapped_coords:
      axes[i, 3].plot(point["crop_corrected"][0], point["crop_corrected"][1], 'ro')
    axes[i, 3].set_title(f'grey_scale_image {i+1}')
    axes[i, 3].axis('off')

    axes[i, 4].imshow(augmented_image, cmap='gray')
    for point in mapped_coords:
      axes[i, 4].plot(point["crop_corrected"][0], point["crop_corrected"][1], 'ro')
    axes[i, 4].set_title(f'augmented_image {i+1}')
    axes[i, 4].axis('off')

    axes[i, 5].imshow(augmented_gbr_image)
    for point in mapped_coords:
      axes[i, 5].plot(point["crop_corrected"][0], point["crop_corrected"][1], 'ro')
    axes[i, 5].set_title(f'augmented_image {i+1}')
    axes[i, 5].axis('off')

plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Example coordinate in the crop-corrected image
four_corners_and_centre = [(400, 400), (0,0), (0,800), (800,0), (800,800)]

unmapped_coords = []

# Unmap the coordinates back to the original image
for point in four_corners_and_centre:
  unmapped_coords.append(unmap_coordinates2(
      crop_corrected_coord=point,
      original_size=(1920, 1080),
      fisheye_params=fisheye_params,
      crop_size1=(1500, 1500),
      crop_offset=(85, 180),
      crop_size2=(800, 800)
  ))

print("Unmapped Coordinates:")
print(unmapped_coords)

fig, axes = plt.subplots(len(originals), 5, figsize=(20, 20))

for i, (original_image, fisheye_image, crop_corrected_image, grey_scale_image, augmented_image) in enumerate(zip(original_images, fisheye_crr_images, crop_corrected_images, grey_scale_images, augmented_images)):
    # Map coordinates to each image stage

    axes[i, 0].imshow(augmented_image, cmap='gray')
    for point in four_corners_and_centre:
      axes[i, 0].plot(point[0], point[1], 'ro')
    axes[i, 0].set_title(f'augmented_image {i+1}')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(grey_scale_image, cmap='gray')
    for point in four_corners_and_centre:
      axes[i, 1].plot(point[0], point[1], 'ro')
    axes[i, 1].set_title(f'grey_scale_image {i+1}')
    axes[i, 1].axis('off')

    axes[i, 2].imshow(crop_corrected_image)
    for point in four_corners_and_centre:
      axes[i, 2].plot(point[0], point[1], 'ro')
    axes[i, 2].set_title(f'crop_corrected_image {i+1}')
    axes[i, 2].axis('off')

    axes[i, 3].imshow(fisheye_image)
    for point in unmapped_coords:
      axes[i, 3].plot(point["undistorted"][0], point["undistorted"][1], 'ro')
    axes[i, 3].set_title(f'undistorted_image {i+1}')
    axes[i, 3].axis('off')

    axes[i, 4].imshow(original_image)
    for point in unmapped_coords:
      axes[i, 4].plot(point["original"][0], point["original"][1], 'ro')
    axes[i, 4].set_title(f'original_image {i+1}')
    axes[i, 4].axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.